after making a new environment, the kernel should preferablly be python 3.11.9

Then we can install SIMPApy

In [ ]:
!pip install SIMPApy

# import packages

In [1]:
import pandas as pd
import numpy as np
import glob
import os
import gseapy as gp
import ipywidgets
import SIMPApy as sp

# load dataframes

In [ ]:
hallmark = r"\data\h.all.v2023.1.Hs.symbols.gmt" # gene set to be used (here HALLMARK from MSigDB). Larger gene sets increase processing time.

In [ ]:
rna = pd.read_csv(r'\data\rna.csv', index_col=0)
cnv = pd.read_csv(r'\data\cn.csv', index_col=0)
meth = pd.read_csv(r'\data\meth.csv', index_col=0)
dna = pd.read_csv(r'\data\dna.csv', index_col=0)

# calculate rankings

In [4]:
rnaranks = sp.calculate_ranking(rna, omic='rna')

In [5]:
cnvranks = sp.calculate_ranking(cnv, omic='cnv')

In [6]:
dnaranks = sp.calculate_ranking(dna, omic='dnam')

then concatenate dataframes

In [7]:
rnarankdf = pd.concat({k: v['weighted'] for k, v in rnaranks.items()}, axis=1)

In [8]:
cnvrankdf = pd.concat({k: v['adjusted_weight'] for k, v in cnvranks.items()}, axis=1)

In [9]:
dnarankdf = pd.concat({k: v['weighted'] for k, v in dnaranks.items()}, axis=1)

# SOPA

In [ ]:
rna_nssGSEA = sp.sopa(rnarankdf, hallmark, output_dir=r"\examples\RNA")

2025-03-01 12:52:31,658 [WARNING] Duplicated values found in preranked stats: 0.70% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2025-03-01 12:52:33,067 [WARNING] Duplicated values found in preranked stats: 0.78% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2025-03-01 12:52:34,585 [WARNING] Duplicated values found in preranked stats: 0.72% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2025-03-01 12:52:36,166 [WARNING] Duplicated values found in preranked stats: 0.72% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2025-03-01 12:52:37,864 [WARNING] Duplicated values found in preranked stats: 0.74% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2025-03-01 12:52:39,663 [WARNING] Duplicated values found in preranked stats: 0.77% of genes
The order of those genes will be

In [ ]:
cnv_nssGSEA = sp.sopa(cnvrankdf, hallmark, output_dir=r"\examples\CNV")

2025-10-06 08:16:25,622 [WARNING] Duplicated values found in preranked stats: 98.96% of genes
The order of those genes will be arbitrary, which may produce unexpected results.


2025-10-06 08:16:26,917 [WARNING] Duplicated values found in preranked stats: 98.46% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2025-10-06 08:16:28,201 [WARNING] Duplicated values found in preranked stats: 98.48% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2025-10-06 08:16:29,678 [WARNING] Duplicated values found in preranked stats: 99.09% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2025-10-06 08:16:31,228 [WARNING] Duplicated values found in preranked stats: 99.07% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2025-10-06 08:16:32,917 [WARNING] Duplicated values found in preranked stats: 99.09% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2025-10-06 08:16:34,790 [WARNING] Duplicated values found in preranked stats: 98.87% of genes
The order of those genes w

In [ ]:
dna_nssGSEA = sp.sopa(dnarankdf, hallmark, output_dir=r"\examples\DNA")

# SIMPA

first get file sample ids from a past sopa run

In [ ]:
# get file list
file_list= glob.glob(os.path.join(r'\examples\RNA', '*_gsea_results.csv')) # dir is to be replaced with r"X:\SOPA_results_folder_Location\".
# sample ids
sample_ids = [os.path.basename(f).split('_')[0] for f in file_list]

load sopa directories for 3 omics

In [ ]:
rna_dir = r"\examples\RNA"
cnv_dir = r"\examples\CNV"
dna_dir = r"\examples\DNA"

Run SIMPA

In [ ]:
simpadf = sp.simpa(sample_ids=sample_ids, 
                                 rna_dir=rna_dir,
                                 cnv_dir=cnv_dir,
                                 dna_dir=dna_dir,
                                 output_dir=r"\examples\SIMPA")

Integration done! Results saved in: C:\Users\Hasan\Desktop\Packaging\SIMPApy_package\examples\SIMPA


# SIMPA analyses

load SIMPA results

In [ ]:
simpaloaded = sp.load_simpa(r"\examples\SIMPA")

In [15]:
# keep terms with FDR < 0.05 to have faster processing. This is optional.
sig = simpaloaded[simpaloaded['fdr_bh'] < 0.05]

here we separate the 2 groups for faster and simpler calculation of group-wide differences

In [16]:
tmas, twas = sp.process_multiomics_data(sig, rna, cnv, dna)

if 1 big group of tmas and twas is required, we could potentially use
```
population = pd.concat([tmas,twas], axis=1)
```

to visualize tmas using the 3d box visualization

we use ```create_interactive_plot(df,'some_heading') # we could keep '' for an empty heading```

In [17]:
sp.create_interactive_plot(tmas,'TMAs Example')

In [18]:
tmas # see dataframe structure

,sample_name,Term,gene_name,fdr,rna_value,rna_nes,rna_tag,mpes,cnv_value,cnv_nes,cnv_tag,dna_value,dna_nes,dna_tag,cancer_type,ajcc_pathologic_stage
0,tm10,HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION,ABI3BP,5.230520e-11,0.4820,-1.444291,1,-4.83850,2.0,0.705905,0,3.726889,-1.462966,1,Not Available,Not Available
1,tm10,HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION,ANPEP,5.230520e-11,3.9514,-1.444291,1,-4.83850,2.0,0.705905,0,1.039212,-1.462966,0,Not Available,Not Available
2,tm10,HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION,AREG,5.230520e-11,565.3915,-1.444291,0,-4.83850,2.0,0.705905,0,-0.759412,-1.462966,1,Not Available,Not Available
3,tm10,HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION,BASP1,5.230520e-11,6.3844,-1.444291,1,-4.83850,2.0,0.705905,0,4.635065,-1.462966,0,Not Available,Not Available
4,tm10,HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION,BMP1,5.230520e-11,14.2645,-1.444291,0,-4.83850,1.0,0.705905,0,-0.458580,-1.462966,1,Not Available,Not Available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5101,tm9,HALLMARK_ANDROGEN_RESPONSE,TNFAIP8,1.311255e-04,18.7285,0.956977,0,-2.44949,6.0,1.171469,1,2.895564,-1.955527,0,Not Available,Not Available
5102,tm9,HALLMARK_ANDROGEN_RESPONSE,TSC22D1,1.311255e-04,214.8955,0.956977,1,-2.44949,3.0,1.171469,0,1.458435,-1.955527,0,Not Available,Not Available
5103,tm9,HALLMARK_ANDROGEN_RESPONSE,UBE2J1,1.311255e-04,70.6214,0.956977,0,-2.44949,4.0,1.171469,0,-1.878903,-1.955527,0,Not Available,Not Available
5104,tm9,HALLMARK_ANDROGEN_RESPONSE,XRCC5,1.311255e-04,126.0580,0.956977,0,-2.44949,4.0,1.171469,0,2.102251,-1.955527,0,Not Available,Not Available
